In [1]:
''' 
Script para descargar un archivo de Google Drive y mostrar las primeras 10 líneas
'''

url_view = "https://drive.google.com/file/d/1RMKdCJKS7vFBhiTLosCotau1k4UsN5iu/view?usp=sharing"   # Enlace de Google Drive
file_id = url_view.split('/')[5]  # Extrae el ID del archivo (la 6ª parte del enlace)
download_url = f"https://drive.google.com/uc?id={file_id}"   # Crea el enlace de descarga directa

import requests
response = requests.get(download_url)

# Muestra las primeras 10 líneas, para saber el tipo de separación del csv
for i, line in enumerate(response.text.splitlines()[:10], start=1):
    print(f"{i:02d}: {line}")


01: Date;Year;Type;Country;State;Location;Activity;Name;Sex;Age;Injury;Fatal Y/N;Time;Species ;Source;pdf;href formula;href;Case Number;Case Number;original order;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
02: 11th October;2025;Unprovoked;Australia;Queensland;Cook Esplanade Thursday Island;Fishing/swimming;Samuel Nai;M;14;Serious abdonminal injuries;N;1823 hrs;Tiger or Bull shark;Kevin McMurray Trackingsharks.com;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;;
03: 7th October;2025;Unprovoked;Australia;South Australia;Kangaroo Island;Surfing;Lee Berryman;M;50+;Lacerations to calf ;N;1330hrs;Bronze whaler?;Kevin

In [2]:
import pandas as pd
from typing import Optional, Iterable

shark_df = pd.read_csv(download_url, sep=';', encoding='utf-8', low_memory=False)
print("Inicialmente los datos tienen:", shark_df.shape)
print(shark_df.head())  # Muestra las primeras filas del DataFrame

shark_df = shark_df.dropna(axis=1, how='all') # Eliminar columnas vacías enteras
print("Ahora los datos tienen:",shark_df.shape)
print(shark_df.columns)  # Muestra las columnas del DataFrame: 255 col a 23 col

Inicialmente los datos tienen: (39139, 255)
             Date  Year        Type     Country            State  \
0    11th October  2025  Unprovoked   Australia       Queensland   
1     7th October  2025  Unprovoked   Australia  South Australia   
2  29th September  2025  Unprovoked         USA   Off California   
3  27th September  2025    Provoked  Costa Rica              NaN   
4   6th September  2025  Unprovoked   Australia              NSW   

                         Location               Activity                Name  \
0  Cook Esplanade Thursday Island       Fishing/swimming          Samuel Nai   
1                 Kangaroo Island                Surfing        Lee Berryman   
2                 Catalina Island               Swimming  Christopher Murray   
3                   Cocos Islands  Diving-Tagging sharks  Dr. Mauricio Hoyos   
4                Long Reef Sydney                Surfing  Mercury Psillaskis   

  Sex  Age  ... Unnamed: 245 Unnamed: 246 Unnamed: 247 Unnamed: 24

In [3]:
#Quitamos las columnas que no nos sirven para el análisis

shark_df.drop(['Date','Location','Name','Age','Injury','Time','Source', 'pdf', 'href formula', 'href', 'Case Number', 'Case Number.1',
       'original order', 'Unnamed: 21', 'Unnamed: 22'], axis=1, inplace=True)

print(shark_df.columns)  # Muestra las columnas del DataFrame después de eliminar las que no analizaremos

Index(['Year', 'Type', 'Country', 'State', 'Activity', 'Sex', 'Fatal Y/N',
       'Species '],
      dtype='object')


In [4]:
""""
Normalize text columns (strip spaces, set lowercase).

Parameters
----------
df : DataFrame
cols : columns to normalize; if None, all object/string columns
lower : convert to lowercase
strip : strip leading/trailing whitespace
normalize_columns: normalize column names (strip, lower, replace spaces with _)

Returns
-------
DataFrame (same object, modified in place style but returns df for chaining)
"""

def standardize_text(
    df: pd.DataFrame,
    cols: Optional[Iterable[str]] = None,
    lower: bool = True,
    strip: bool = True,
    normalize_columns: bool = True,
) -> pd.DataFrame:
       
    if cols is None:
        cols = df.select_dtypes(include=["object", "string"]).columns

    for c in cols:
        s = df[c].astype("string")
        if strip:
            s = s.str.strip()
        if lower:
            s = s.str.lower()
        df[c] = s
     
    if normalize_columns:
        df.columns = (
            df.columns
            .str.strip()
            .str.lower()
            .str.replace(" ", "_")
        )

    return df

shark_df = standardize_text(shark_df)
print(shark_df.head())  # Muestra las primeras filas del DataFrame después de la normalización del texto y las columnas

   year        type     country            state               activity sex  \
0  2025  unprovoked   australia       queensland       fishing/swimming   m   
1  2025  unprovoked   australia  south australia                surfing   m   
2  2025  unprovoked         usa   off california               swimming   m   
3  2025    provoked  costa rica             <NA>  diving-tagging sharks   m   
4  2025  unprovoked   australia              nsw                surfing   m   

  fatal_y/n              species  
0         n  tiger or bull shark  
1         n       bronze whaler?  
2         n   unknown 1.2m shark  
3         n       tiger shark 4m  
4         y    great white shark  


In [5]:
#Check valores únicos por columna:

for col in shark_df.columns:
    print(f'{col.upper()} --> valores únicos:', shark_df[col].unique())

YEAR --> valores únicos: <StringArray>
['2025', '2024', '2026', '2023', '2022', '2021', '2020', '2019', '2018',
 '2017',
 ...
 '1580', '1555', '1554', '1543', '1518', '1500', '1000', '0077', '0005',
 '0000']
Length: 263, dtype: string
TYPE --> valores únicos: <StringArray>
[         'unprovoked',            'provoked',        'questionable',
          'watercraft',        'sea disaster',                  <NA>,
                   '?',         'unconfirmed',          'unverified',
             'invalid', 'under investigation',                'boat']
Length: 12, dtype: string
COUNTRY --> valores únicos: <StringArray>
[               'australia',                      'usa',
               'costa rica',                  'bahamas',
              'puerto rico',         'french polynesia',
                    'spain',           'canary islands',
             'south africa',                  'vanuatu',
 ...
        'mediterranean sea',                   'sweden',
                   'roatan', 'b

In [6]:
shark_df.dtypes

year         string[python]
type         string[python]
country      string[python]
state        string[python]
activity     string[python]
sex          string[python]
fatal_y/n    string[python]
species      string[python]
dtype: object

In [7]:
print('Los valores iniciales eran:', shark_df['year'].nunique)
print('Los valores iniciales eran:', shark_df['year'].shape)
print('Los valores iniciales eran:', shark_df['type'].nunique)
print('Los valores iniciales eran:', shark_df['type'].shape)

#PRIMER FILTRO (Marlene):

shark_df = shark_df[shark_df['type'] == 'unprovoked'].copy() #Quitamos los accidentes que no sean "unprovoked"

Los valores iniciales eran: <bound method IndexOpsMixin.nunique of 0        2025
1        2025
2        2025
3        2025
4        2025
         ... 
39134    <NA>
39135    <NA>
39136    <NA>
39137    <NA>
39138    <NA>
Name: year, Length: 39139, dtype: string>
Los valores iniciales eran: (39139,)
Los valores iniciales eran: <bound method IndexOpsMixin.nunique of 0        unprovoked
1        unprovoked
2        unprovoked
3          provoked
4        unprovoked
            ...    
39134          <NA>
39135          <NA>
39136          <NA>
39137          <NA>
39138          <NA>
Name: type, Length: 39139, dtype: string>
Los valores iniciales eran: (39139,)


In [ ]:
#Exportamos a excel post filtro: TYPE

shark_df.to_excel('SharkAttack_check1.xlsx')

In [ ]:
print(shark_df[['year']].isnull().sum())

shark_df = shark_df.dropna(subset=['year']) #Quitamos los null de la columna "year"

print(shark_df.isnull().sum())

year    0
dtype: int64
year            0
type            0
country        33
state         300
activity      359
sex           168
fatal_y/n      17
species      2598
dtype: int64


In [ ]:
#formateamos para que todos los string sean igual (con decimal ".")

shark_df['year'] = (
    shark_df['year']
    .astype(str)                 
    .str.replace(',', '.', regex=False)  # cambia coma por punto
    .astype(float)      
)
shark_df['year'] = pd.to_numeric(shark_df['year'])
shark_df = shark_df.loc[(shark_df['year'] >= 2000) & (shark_df['year'] <= 2025)] #filtramos los años sujetos a estudio

#CHECK FILTRO 1
print('Post filtro quedan:', shark_df['year'].shape , "valores")
print('Post filtro quedan:', shark_df['type'].shape , "valores")


Post filtro quedan: (2206,) valores
Post filtro quedan: (2206,) valores


In [15]:
#Check valores por columnas para próximos filtros

for col in shark_df.columns:
    print(f'{col.upper()} --> valores únicos:', shark_df[col].unique())

YEAR --> valores únicos: [2025. 2024. 2023. 2022. 2021. 2020. 2019. 2018. 2017. 2016. 2015. 2014.
 2013. 2012. 2011. 2010. 2009. 2008. 2007. 2006. 2005. 2004. 2003. 2002.
 2001. 2000.]
TYPE --> valores únicos: <StringArray>
['unprovoked']
Length: 1, dtype: string
COUNTRY --> valores únicos: <StringArray>
[                            'australia',
                                   'usa',
                               'bahamas',
                           'puerto rico',
                      'french polynesia',
                                 'spain',
                        'canary islands',
                          'south africa',
                               'vanuatu',
                               'jamaica',
                                'israel',
                              'maldives',
                      'turks and caicos',
                            'mozambique',
                         'new caledonia',
                                 'egypt',
                      

In [16]:
#Exportamos a excel post filtro: TYPE + YEAR

shark_df.to_excel('SharkAttack_check2.xlsx')